# 주소 정리

## library

In [2]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

## 데이터 로드

In [3]:
df = pd.read_csv('2020-22년_급속충전.csv', index_col=0, usecols=['충전소명', '주소', '지역', '시군구'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11665821 entries, 내린천휴게소 양양방향 to 마천동2공영주차장
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   지역      object
 1   시군구     object
 2   주소      object
dtypes: object(3)
memory usage: 356.0+ MB


In [4]:
df = df.reset_index()
df.head()

,충전소명,지역,시군구,주소
0,내린천휴게소 양양방향,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
1,내린천휴게소 양양방향,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
2,내린천휴게소 양양방향,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
3,내린천휴게소 양양방향,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
4,한국광기술원,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9


In [5]:
adr_uni = df['주소'].unique()
len(adr_uni)

4365

In [6]:
len(df['충전소명'].unique())

4217

In [7]:
df2 = df.drop_duplicates(['주소']).reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4365 entries, 0 to 4364
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   충전소명    4365 non-null   object
 1   지역      4365 non-null   object
 2   시군구     4365 non-null   object
 3   주소      4365 non-null   object
dtypes: object(4)
memory usage: 136.5+ KB


In [8]:
len(df2['충전소명'].unique())

4161

In [9]:
adr = df2['주소'].to_numpy()
len(adr)

4365

### 특이주소 처리

In [10]:
# 경상북도 경주시 천원2길 9 경북 경주시 교동 153-5
# 충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1
# 광주광역시 북구 첨단 벤처로 108번길 9

In [11]:
count = 0
for i, one in enumerate(adr):
    if '경상북도 경주시 천원2길 9 경북 경주시 교동 153-5' in one:
        adr[i] = '경상북도 경주시 교동 천원2길 9'
        count += 1
count

1

In [12]:
count = 0
for i, one in enumerate(adr):
    if '충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1' in one:
        adr[i] = '충청북도 단양군 영춘면 온달평강3로 9'
        count += 1
count

1

In [13]:
count = 0
for i, one in enumerate(adr):
    if '광주광역시 북구 첨단 벤처로 108번길 9' in one:
        adr[i] = '광주광역시 북구 첨단벤처로108번길 9'
        count += 1
count

1

### 빈칸 2개 -> 1개

In [14]:
count = 0
for i, one in enumerate(adr):
    if '  ' in one:
        adr[i] = one.replace('  ', ' ')
        count += 1
count

128

### 컴마(,) 제거

In [15]:
count = 0
for i, one in enumerate(adr):
    if ',' in one:
        adr[i] = one.replace(',', '')
        count += 1
count

324

### 복수 주소 오류
- 한주소에 2개 주소 들어있는 오류
- 동일지역 : 56
- 다른지역 : 2

#### 동일지역

In [16]:
np.where(adr == '전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9')

(array([2609], dtype=int64),)

In [17]:
adr[2608]

'충청남도 청양군 청양읍 군량리 533 충청남도 태안군 소원면 신덕리 272'

In [18]:
si = df2['시군구'].to_numpy()

In [19]:
count = 0
for i, one in tqdm(enumerate(adr)):
    first = list(one.split())[0]
    one = one.replace(' ' + first, '$' + first)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
        for j, two in enumerate(multi_adr):
            if si[i] in two:
                adr[i] = multi_adr[j]
count

4365it [00:00, 336645.65it/s]


56

In [20]:
adr[2608]

'충청남도 청양군 청양읍 군량리 533'

#### 다른지역

In [21]:
sep = ' 광주광역시| 전라남도| 전라북도| 제주특별자치도| 세종특별자치시| 서울특별시| 대전광역시| 경기도| 인천광역시| 경상남도| 강원도| 충청북도| 경상북도| 충청남도| 울산광역시| 대구광역시| 부산광역시'

In [22]:
count = 0
for i, one in tqdm(enumerate(adr)):
    multi_local = list(re.split(sep, one))
    if len(multi_local) == 2:
        count += 1
count

4365it [00:00, 312638.95it/s]


2

In [23]:
rep_dict = {" 광주광역시":"$광주광역시" ," 전라남도":"$전라남도"," 전라북도":"$전라북도"," 제주특별자치도":"$제주특별자치도"," 세종특별자치시":"$세종특별자치시"," 서울특별시":"$서울특별시"," 대전광역시":"$대전광역시"," 경기도":"$경기도"," 인천광역시":"$인천광역시"," 경상남도":"$경상남도"," 강원도":"$강원도"," 충청북도":"$충청북도"," 경상북도":"$경상북도"," 충청남도":"$충청남도"," 울산광역시":"$울산광역시"," 대구광역시":"$대구광역시"," 부산광역시":"$부산광역시"}

In [24]:
np.where(adr == '전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2')

(array([2457], dtype=int64),)

In [25]:
adr[2456]

'전라북도 진안군 안천면 삼락리 1102-5'

In [26]:
count = 0
for i, one in tqdm(enumerate(adr)):
    for k, v in rep_dict.items():
        one = one.replace(k, v)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
        for j, two in enumerate(multi_adr):
            if si[i] in two:
                adr[i] = multi_adr[j]
count

4365it [00:00, 131000.72it/s]


2

In [27]:
adr[2456]

'전라북도 진안군 안천면 삼락리 1102-5'

In [28]:
adr[44]

'서울특별시 성동구 광나루로4가길 23'

### 도로명 주소 정리
- [길 숫자] 분리
- [로 숫자] 분리
- -문자 (숫자)(번)길 붙이기
    - [로 숫자(번)길] 붙이기
    - [앙 (숫자)길] 붙이기
    - [장 (숫자)길] 붙이기
    - [솜 (숫자)로] 붙이기
    - [길 (숫자)번길] 붙이기
    - [터 (숫자)길] 붙이기
    - [개 (숫자)길] 붙이기

#### [분리] [숫자]-길|-로|-번길 [숫자]
- 주의
    - 분리 YES
        - 834, 경상남도 거창군 거창읍 심소정길39-36
        - 1201, 강원도 삼척시 수로부인길453
    - 분리 NO
        - 743, 서울특별시 영등포구 신길1동 116-15
        - 3673, 경상북도 포항시 남구 구룡포읍 구룡포길117번길 28-8

In [29]:
np.where(adr == '전라북도 전주시 완산구 신촌3길24번지')

(array([15], dtype=int64),)

In [30]:
adr[15]

'전라북도 전주시 완산구 신촌3길24번지'

In [31]:
s = adr[15]
split1 = re.compile('([가-힣]+[0-9]+[길|로|번길])([0-9]+)')
split1.sub('\g<1> \g<2>', s)

'전라북도 전주시 완산구 신촌3길 24번지'

In [32]:
adr[743]

'서울특별시 영등포구 신길1동 116-15'

In [33]:
s = adr[743]
split1.sub('\g<1> \g<2>', s)

'서울특별시 영등포구 신길1동 116-15'

In [34]:
adr[3673]

'충청남도 예산군 예산읍 신례원리 611-151'

In [35]:
s = adr[3673]
split1.sub('\g<1> \g<2>', s)

'충청남도 예산군 예산읍 신례원리 611-151'

In [36]:
count = 0
idx = []
for i, one in enumerate(adr):
    if split1.search(one):
        print(i, one)
        idx.append(i)
        count += 1
count

15 전라북도 전주시 완산구 신촌3길24번지
120 충청북도 제천시 의림대로42길34
547 서울특별시 송파구 오금로68길8
692 경상남도 김해시 번화1로56번길 15 롯데마트
1000 제주특별자치도 제주시 조천읍 선흘서5길7
1342 충청북도 괴산군 청안면 청안읍내로2길10
1675 충청북도 괴산군 불정면 목도로2길11
1854 강원도 철원군 동송읍 이평2로17번길32
1951 충청북도 괴산군 장연면 미선로오가5길7
2162 서울특별시 강서구 화곡로65길62
2971 경상남도 합천군 합천읍 교동1길33
2980 충청북도 괴산군 칠성면 명태재로외사4길1
2995 경상남도 통영시 광도면 죽림2로45
3000 경상남도 창녕군 길곡면 증산2길19
3009 경상북도 칠곡군 동명면 남원로5길77
3015 경상북도 경산시 진량읍 공단7로126
3039 충청남도 서산시 고북면 고북1로294
3052 충청남도 천안시 동남구 풍세산단4로81
3070 경상남도 창원시 마산합포구 현동7길27
3072 충청남도 계룡시 엄사면 문화1로13
3079 대구광역시 동구 첨단로8길32
3080 충청남도 논산시 산양1길2
3082 충청남도 부여군 규암면 아름1로64
3153 충청북도 영동군 황간면 원촌동1길54-8
3166 경상북도 봉화군 춘양면 의양로5길19
3193 경상북도 청송군 진보면 이촌1길23-1
3325 전라북도 군산시 미장8길7
3397 울산광역시 울주군 청량읍 처용산업4길51
3409 경상북도 구미시 산동면 첨단기업1로17
3439 경상북도 칠곡군 왜관읍 평장3길47


30

In [37]:
len(idx)

30

In [38]:
count = 0
idx = []
for i, one in enumerate(adr):
    if split1.search(one):
        adr[i] = split1.sub('\g<1> \g<2>', one)
        count += 1
count

30

In [39]:
adr[[15, 743, 3673]]

array(['전라북도 전주시 완산구 신촌3길 24번지', '서울특별시 영등포구 신길1동 116-15',
       '충청남도 예산군 예산읍 신례원리 611-151'], dtype=object)

#### [분리] -길-로-번길 [숫자]-[숫자]

In [40]:
adr[834]

'경상남도 거창군 거창읍 심소정길39-36'

In [41]:
s = adr[834]
split2 = re.compile('([가-힣]+[길|로|번길])([0-9]+[-][0-9]+)')
split2.search(s)

<re.Match object; span=(13, 22), match='심소정길39-36'>

In [42]:
split2.sub('\g<1> \g<2>', s)

'경상남도 거창군 거창읍 심소정길 39-36'

In [43]:
adr[1201]

'강원도 삼척시 수로부인길453'

In [44]:
s = adr[1201]
split2 = re.compile('([가-힣]+[길|로|번길])([0-9]+[-][0-9]+)')
split2.sub('\g<1> \g<2>', s)

'강원도 삼척시 수로부인길453'

In [45]:
count = 0
idx = []
for i, one in enumerate(adr):
    if split2.search(one):
        adr[i] = split2.sub('\g<1> \g<2>', one)
        count += 1
count

27

In [46]:
adr[[834, 1201]]

array(['경상남도 거창군 거창읍 심소정길 39-36', '강원도 삼척시 수로부인길453'], dtype=object)

#### [분리] -길-로-번길 [(순수)숫자] 분리

In [47]:
split3 = re.compile('([가-힣]+[길|로|번길])([0-9]+)')
split31 = re.compile('([가-힣]+[0-9]+[길|로|번길]) ([0-9]+)')

In [48]:
adr[1201]

'강원도 삼척시 수로부인길453'

In [49]:
s = adr[1201]
if split3.search(s) and not split31.search(s):
    print(s)

강원도 삼척시 수로부인길453


In [50]:
adr[834]

'경상남도 거창군 거창읍 심소정길 39-36'

In [51]:
s = adr[834]
if split3.search(s) and not split31.search(s):
    print(s)

In [52]:
adr[1]

'광주광역시 북구 첨단벤처로108번길 9'

In [53]:
s = adr[1]
if split3.search(s) and not split31.search(s):
    print(s)

광주광역시 북구 첨단벤처로108번길 9


In [54]:
count = 0
idx3 = []
for i, one in enumerate(adr):
    if split3.search(one):
        print(i, one)
        idx3.append(i)
        count += 1

1 광주광역시 북구 첨단벤처로108번길 9
4 전라남도 구례군 산동면 노고단로1068
18 제주특별자치도 서귀포시 성산읍 해맞이해안로2660
35 대전광역시 유성구 반석로12번길 18-4
43 서울특별시 마포구 마포대로1길 9
44 서울특별시 성동구 광나루로4가길 23
65 충청북도 충주시 으뜸로21
79 제주특별자치도 제주시 중앙로25길 17
85 경기도 이천시 마장면 중부고속도로82
88 경기도 용인시 처인구 주북로94번길 30-1 (고림동)
100 경기도 시흥시 서울대학로278번길 67 지상4층 주차장 A4
101 경상남도 양산시 백호로23 축구장 옆
103 서울특별시 영등포구 의사당대로1 국회의원회관 주차장 우측
116 경기도 가평군 설악면 미사리로540번길 51
120 충청북도 제천시 의림대로42길 34
131 경기도 화성시 동탄공원로2길 32
138 제주특별자치도 서귀포시 소보리당로164번길 62  주차장 좌측
177 대구광역시 동구 아양로37길 92
178 대구광역시 북구 복현로8길 16-5
215 경기도 이천시 마장면 중부대로609번길 244-35 (목리)
216 경기도 이천시 신둔면 서이천로853번길 346-129 (용면리)
232 전라남도 곡성군 곡성읍 학교로8
296 경상남도 김해시 장유로116번길 44-82 (부곡동)
316 경기도 동두천시 평화로2910번길 30
322 경기도 양평군 청운면 용두로164번길 9-5
324 경기도 이천시 마장면 덕이로154번길 287-76 휴게소 옥외 장애인 주차장(휴게소 정면 우측 흡연장 앞)
326 경기도 군포시 군포로609번안길 12
369 서울특별시 강동구 진황도로29길 34 지하1층 26
374 서울특별시 관악구 난곡로34나길 14 2층
375 서울특별시 관악구 낙성대로4가길 5
379 서울특별시 노원구 동일로204가길 12 5층 출구앞
402 서울특별시 영등포구 의사당대로1 국회도서관 앞
409 강원도 삼척시 청석로3길 16-41
421 제주특별자치도 서귀포시 표선면 성읍정의현로22번길 7-1  

371

In [62]:
count

371

In [55]:
# 4 전라남도 구례군 산동면 노고단로1068
# 18 제주특별자치도 서귀포시 성산읍 해맞이해안로2660
# 65 충청북도 충주시 으뜸로21
# 85 경기도 이천시 마장면 중부고속도로82
# 101 경상남도 양산시 백호로23 축구장 옆
# 465 경상북도 성주군 성주읍 성주순환로251
# 543 서울특별시 강남구 남부순환로3104
# 559 제주특별자치도 제주시 한림읍 금악서길133
# 565 경상북도 청송군 파천면 청송로5544
# 665 경상북도 포항시 북구 해안로82
# 693 서울특별시 강남구 테헤란로606
## 717 서울특별시 송파구 성내천로33가길20
# 770 강원도 철원군 갈말읍 삼부연로38
# 905 서울특별시 영등포구 양산로111  왼쪽 끝편 주차장
# 995 제주특별자치도 서귀포시 상효로39 (상효동 754-1)
# 1057 경상남도 창원시 의창구 창원대로314
# 1192 경상북도 의성군 안계면 서부로1783
# 1201 강원도 삼척시 수로부인길453
# 1205 강원도 원주시 입춘로65
# 1324 전라남도 영광군 백수읍 해안로918
## 1332 충청북도 청주시 흥덕구 직지대로620번길70
# 1338 충청북도 청주시 서원구 수곡로61
# 1341 충청북도 괴산군 연풍면 향교로71
# 1505 경상남도 양산시 어곡터널로40
# 1506 경상남도 양산시 물금읍 황산로501
# 1522 서울특별시 양천구 목동동로81
# 1526 강원도 평창군 진부면 경강로3562
# 1566 경상북도 상주시 화남면 문장로758
# 1569 경상북도 포항시 남구 장기면 동해안로2987
# 1581 경기도 의정부시 동일로30
# 1606 경상북도 영천시 천문로594
# 1619 경상북도 경산시 진량읍 공단로299
# 1766 강원도 삼척시 새천년도로673
# 1768 대전광역시 대덕구 신탄진로556
# 1777 경기도 양주시 은현면 은남로177
# 1783 전라남도 영광군 천년로1329
# 1844 경상북도 영천시 한방로77
# 1850 전라북도 남원시 운봉읍 황산로1026
### 1854 강원도 철원군 동송읍 이평2로 17번길32
# 1871 경기도 시흥시 시흥대로869
# 1914 울산광역시 북구 산업로1103
# 1923 경기도 시흥시 동서로425
# 1943 충청남도 논산시 채운면 채운로41
# 1983 서울특별시 노원구 노원로75
# 2004 경상북도 상주시 영남제일로1950
# 2056 전라남도 영광군 영광읍 영광로1330
# 2058 울산광역시 울주군 웅촌면 새초천길12
# 2085 인천광역시 서구 중봉대로211
# 2089 충청북도 진천군 진광로487
# 2190 광주광역시 광산구 풍영로179
# 2328 경기도 군포시 산본로267
# 2586 경상북도 의성군 비안면 강변길59
# 2627 경상북도 경주시 산내면 문복로1822
# 2694 경상북도 포항시 장기면 양포항길12
# 2704 경상남도 진주시 금산면 송백로46
## 2747 부산광역시 강서구 공항진입로42번길20
# 2780 제주특별자치도 제주시 용두암길15
# 2957 충청남도 부여군 충화면 팔충로770
# 2958 경상북도 구미시 선산읍 김선로963
# 2959 경상남도 합천군 합천읍 대야로991
# 2960 경상남도 창녕군 창녕읍 탐하로201
# 2962 충청남도 보령시 대천로113
# 2965 충청북도 음성군 맹동면 원중로1339
# 2969 경상북도 구미시 도개면 도안로103
# 2971 경상남도 의령군 유곡면 청정로1679
# 2973 경상남도 창녕군 성산면 성산로237
# 2974 경상북도 구미시 박정희로107
# 2975 경상남도 합천군 적중면 적중로98
# 2976 울산광역시 울주군 삼동면 보삼길550
# 2978 충청북도 영동군 용산면 영동산단로27
# 2980 경상북도 문경시 중앙로50
# 2984 충청북도 영동군 매곡면 민주지산로3595
# 2988 전라남도 영암군 삼호읍 공항로21
# 2990 강원도 강릉시 성산면 삼포암길133
## 2993 경상남도 창녕군 유어면 우포1대로583
# 2995 경상남도 합천군 용주면 황계폭포로1154
# 2996 경상북도 청송군 주왕산면 부동로1008
# 3010 경상북도 청송군 파천면 안파로1244
# 3011 전라남도 장흥군 용산면 장흥대로2667
# 3013 경상남도 거창군 신원면 신차로2924
# 3016 경상북도 성주군 성주읍 주산로193
## 3017 충청남도 부여군 양화면 입포로53번길6
# 3021 경상북도 구미시 경부고속도로172
# 3030 경상북도 봉화군 봉성면 부랭이길88
# 3032 경상남도 창원시 의창구 소계로135
# 3033 경상남도 창원시 진해구 웅천중로59
# 3034 경상북도 청송군 진보면 청송로6359
# 3040 경상북도 영양군 영양읍 중앙로55
## 3042 인천광역시 중구 신포로27번길80
# 3043 전라남도 보성군 용문길18
# 3044 충청남도 청양군 비봉면 충절로1994
# 3049 충청남도 서산시 문화로112
# 3053 경상북도 봉화군 상운면 예봉로1281
# 3056 충청남도 부여군 은산면 충의로694
# 3059 전라남도 보성군 미력면 가평길44
# 3060 경기도 파주시 시청로66
# 3061 경상북도 영양군 영양읍 군청길37
# 3070 경상북도 영주시 풍기읍 소백로2156
### 3073 대전광역시 유성구 송림로 19번길35
# 3074 인천광역시 남동구 남동서로209
# 3080 경상북도 청송군 청송읍 주왕산로177
# 3083 경상남도 합천군 초계면 동부로1240
# 3084 충청남도 보령시 천북면 홍보로462
# 3085 경상남도 의령군 부림면 입산로178
# 3090 경상북도 청송군 청송읍 군청로25
# 3093 대전광역시 유성구 신성로55
# 3096 경상북도 구미시 형곡로180
# 3098 경상남도 의령군 낙서면 낙동강로859
# 3100 경상북도 안동시 관광단지로316
### 3101 대전광역시 유성구 노은동로 87번길89
# 3132 충청남도 예산군 삽교읍 예학로65
# 3169 경상북도 청송군 안덕면 방호정로50
# 3177 경상북도 청송군 주왕산면 이전앞길27
# 3186 경상북도 청송군 안덕면 안덕로82
# 3187 광주광역시 북구 하서로111
# 3189 경상북도 영양군 영양읍 군민회관길46
# 3194 경상북도 청송군 청송읍 중앙로184
# 3198 충청북도 제천시 세명로21
# 3199 대전광역시 유성구 유성대로1719
# 3211 충청북도 영동군 용화면 휴양림길60
# 3216 경기도 화성시 송산면 평택시흥고속도로25
## 3224 경기도 시흥시 시흥대로268번길8
# 3240 전라북도 정읍시 내장호반로266
# 3252 전라북도 익산시 왕궁면 궁성로666
# 3272 강원도 춘천시 칠전동길28
# 3273 전라북도 군산시 창길21
# 3280 강원도 정선군 사북읍 사북중앙로65
# 3297 경기도 구리시 장자대로141
# 3363 경상북도 영주시 대학로352
# 3364 경기도 화성시 팔탄면 서해안고속도로302
# 3365 경기도 용인시 수지구 경부고속도로400
# 3372 경상남도 밀양시 산내면 얼음골로177
# 3373 경상북도 의성군 사곡면 의성사곡로966
# 3378 경상북도 의성군 안사면 안신로645
## 3387 전라남도 목포시 남악로162번길25
# 3399 전라남도 보성군 회천면 우암길21
# 3401 충청남도 금산군 금성면 의총길25
# 3409 전라북도 남원시 이백로13
# 3413 경상북도 구미시 광평길55
## 3419 경상남도 창원시 의창구 태복산로15번길8
# 3422 경상북도 영주시 영주로204
## 3424 경상남도 진주시 가좌길29번길63
# 3427 경상북도 문경시 중앙로280
# 3430 경상남도 밀양시 산내면 얼음골로334
# 3431 경상북도 구미시 송정대로89
# 3432 경상북도 구미시 해마루공원로24
# 3435 경상북도 울진군 온정면 백암온천로1308
# 3436 경상북도 예천군 효자면 은풍로1154
# 3439 경상북도 울진군 온정면 백암온천로1185
# 3470 광주광역시 서구 천변우하로391
# 3518 경상남도 창원시 마산합포구 진동면 신서촌로15
# 3550 경상북도 안동시 학가산온천길42
# 3669 충청북도 청주시 서원구 흥덕로48
# 3694 충청남도 아산시 실옥로234
# 3700 광주광역시 북구 추암로249
### 3742 경상남도 창원시 의창구 사림로 99번길63
# 3886 울산광역시 북구 천곡남로86
# 3887 울산광역시 남구 갈밭로49

In [56]:
s = adr[101]
split4 = re.compile('([가-힣]+)([0-9]+)')
split4.sub('\g<1> \g<2>', s)

'경상남도 양산시 백호로 23 축구장 옆'

In [57]:
adr[103]

'서울특별시 영등포구 의사당대로1 국회의원회관 주차장 우측'

In [64]:
s = adr[103]
split4 = re.compile('([가-힣]+)([0-9]+)')
split4.sub('\g<1> \g<2>', s, 1)

'서울특별시 영등포구 의사당대로 1 국회의원회관 주차장 우측'

In [65]:
adr[771]

'강원도 철원군 서면 와수리1160-3'

In [66]:
s = adr[771]
split4 = re.compile('([가-힣]+)([0-9]+)')
split4.sub('\g<1> \g<2>', s, 1)

'강원도 철원군 서면 와수리 1160-3'

In [77]:
idx2 = [4, 18, 65, 85, 101, 465, 543, 559, 565, 665, 
        693, 770, 905, 995, 1057, 1192, 1201, 1324, 1338, 1341,
        1505, 1506, 1522, 1526, 1566, 1569, 1581, 1606, 1619, 1766,
        1768, 1777, 1783, 1844, 1850, 1871, 1914, 1923, 1943, 1983,
        2004, 2056, 2058, 2085, 2089, 2190, 2328, 2586, 2627, 2694,
        2704, 2780, 2957, 2958, 2959, 2960, 2962, 2965, 2969, 2971,
        2973, 2974, 2975, 2976, 2978, 2980, 2984, 2988, 2990, 2995,
        2996, 3010, 3011, 3013, 3016, 3021, 3030, 3032, 3033, 3034,
        3040, 3043, 3044, 3049, 3053, 3056, 3059, 3060, 3061, 3070, 
        3074, 3080, 3083, 3084, 3085, 3090, 3093, 3096, 3098, 3100,
        3132, 3169, 3177, 3186, 3187, 3189, 3194, 3198, 3199, 3211,
        3216, 3240, 3252, 3272, 3273, 3280, 3297, 3363, 3364, 3365,
        3372, 3373, 3378, 3399, 3401, 3409, 3413, 3422, 3427, 3430,
        3431, 3432, 3435, 3436, 3439, 3470, 3518, 3550, 3669, 3694,
        3700, 3886, 3887, 103, 402, 771, 1205, 1507, 1620, 1639,
        1670, 1709, 1765, 1819, 1905, 2104, 2170, 2443, 3041, 3066,
        3067, 3092, 3174, 3541, 3551, 3726, 3017, 3042]

In [78]:
for i in idx2:
    temp = adr[i]
    adr[i] = split4.sub('\g<1> \g<2>', temp, 1)

In [79]:
adr[[4, 101, 103, 771]]

array(['전라남도 구례군 산동면 노고단로 1068', '경상남도 양산시 백호로 23 축구장 옆',
       '서울특별시 영등포구 의사당대로 1 국회의원회관 주차장 우측', '강원도 철원군 서면 와수리 1160-3'],
      dtype=object)

In [94]:
s = '서울특별시 송파구 성내천로33가길20'
split5 = re.compile('([가-힣]+[로길][0-9]+[가번][길])([0-9]+)')
split5.search(s)

<re.Match object; span=(10, 20), match='성내천로33가길20'>

In [97]:
split5.sub('\g<1> \g<2>', s)

'서울특별시 송파구 성내천로33가길 20'

In [96]:
idx3 = []
for i, one in enumerate(adr):
    if split5.search(one):
        idx3.append(i)
        print(i, one)

717 서울특별시 송파구 성내천로33가길20
1332 충청북도 청주시 흥덕구 직지대로620번길70
2748 부산광역시 강서구 공항진입로42번길20
3018 충청남도 부여군 양화면 입포로53번길6
3225 경기도 시흥시 시흥대로268번길8
3388 전라남도 목포시 남악로162번길25
3420 경상남도 창원시 의창구 태복산로15번길8
3425 경상남도 진주시 가좌길29번길63


In [99]:
idx3

[717, 1332, 2748, 3018, 3225, 3388, 3420, 3425]

In [100]:
type(idx3[1])

int

In [101]:
for i in idx3:
    temp = adr[i]
    adr[i] = split5.sub('\g<1> \g<2>', temp)

In [102]:
adr[idx3]

array(['서울특별시 송파구 성내천로33가길 20', '충청북도 청주시 흥덕구 직지대로620번길 70',
       '부산광역시 강서구 공항진입로42번길 20', '충청남도 부여군 양화면 입포로53번길 6',
       '경기도 시흥시 시흥대로268번길 8', '전라남도 목포시 남악로162번길 25',
       '경상남도 창원시 의창구 태복산로15번길 8', '경상남도 진주시 가좌길29번길 63'], dtype=object)

In [104]:
s = '강원도 철원군 동송읍 이평2로 17번길32'
split6 = re.compile('([가-힣]+[0-9]*로) ([0-9]+번길)([0-9]+)')
split6.sub('\g<1>\g<2> \g<3>', s)

'강원도 철원군 동송읍 이평2로17번길 32'

In [106]:
idx4 = []
for i, one in enumerate(adr):
    if split6.search(one):
        idx4.append(i)
        print(i, one)

1854 강원도 철원군 동송읍 이평2로 17번길32
3102 대전광역시 유성구 노은동로 87번길89
3743 경상남도 창원시 의창구 사림로 99번길63


In [107]:
adr[idx4]

array(['강원도 철원군 동송읍 이평2로 17번길32', '대전광역시 유성구 노은동로 87번길89',
       '경상남도 창원시 의창구 사림로 99번길63'], dtype=object)

In [108]:
s = '대전광역시 유성구 노은동로 87번길89'
split6.sub('\g<1>\g<2> \g<3>', s)

'대전광역시 유성구 노은동로87번길 89'

In [109]:
s = '강원도 철원군 동송읍 이평2로 17번길32'
split6.sub('\g<1>\g<2> \g<3>', s)

'강원도 철원군 동송읍 이평2로17번길 32'

In [110]:
for i in idx4:
    temp = adr[i]
    adr[i] = split6.sub('\g<1>\g<2> \g<3>', temp)

In [111]:
adr[idx4]

array(['강원도 철원군 동송읍 이평2로17번길 32', '대전광역시 유성구 노은동로87번길 89',
       '경상남도 창원시 의창구 사림로99번길 63'], dtype=object)

#### [분리] "-동|면|읍" "숫자"

In [115]:
count = 0
split7 = re.compile('([가-힣]+[0-9]*[동면읍])([0-9]+)')
for i, one in enumerate(adr):
    if split7.search(one):
        count += 1
        print(i, one)

16 제주특별자치도 제주시 연동2334-4
163 경상남도 창원시 성산구 사파동64-2
389 충청북도 단양군 적성면 기동2길 32-37 (기동리)
439 전라북도 전주시 완산구 효자동3가 1712-10
443 전라북도 전주시 완산구 효자동2가 1234-3 5층
605 전라북도 전주시 덕진구 인후동1가 807-6
645 경상북도 구미시 인동14길 14
649 대구광역시 북구 노원동2가 214
687 경상남도 창원시 마산합포구 신마산시장길 33 (창포동3가)
854 경상남도 창원시 진해구 용재로 46  웅동2동 주민센터 주차장 내
982 서울특별시 종로구 인사동5길 29 지상주차장
1056 경상남도 창원시 장군동4가 3-6
1062 부산광역시 중구 중앙대로2 (중앙동7가)
1135 충청북도 청주시 서원구 현도면 선동1길 9
1139 충청남도 당진시 무수동2길 25-21
1206 대전광역시 동구 용운동423-22
1368 울산광역시 울주군 언양읍 언동3길 20-48
1377 대전광역시 동구 홍도동92-3
1426 강원도 삼척시 당저동1길 17-9
1568 경상북도 구미시 오태동659-13
1647 강원도 태백시 소도동326
1748 인천광역시 중구 북성동1가 97-32
1811 경상남도 창원시 진해구 명동580 좌측 주차장
2397 울산광역시 남구 선암동584-8번지
2422 전라북도 전주시 덕진구 송천동2가 1257
2424 전라북도 전주시 덕진구 송천동2가 1316
2617 경상남도 창원시 마산회원구 석전동4길 52
2855 인천광역시 중구 송월동2가 4
2863 전라북도 전주시 덕진구 가련산로 26-3 (송천동1가)
3153 충청북도 영동군 황간면 원촌동1길 54-8
3286 전라북도 익산시 인화동2가 7
3454 전라남도 여수시 삼동3길 13
3892 경상남도 고성군 거류면 당동3길 5
3925 전라남도 여수시 학동2길 7
3988 전라남도 영암군 신북면 황금동1길 5-1
4051 충청북도 청주시 서원구 현도면 선동2길 232-9
4182 전라남도 여수시 

In [116]:
count

38

In [114]:
# 16 제주특별자치도 제주시 연동2334-4
# 163 경상남도 창원시 성산구 사파동64-2
# 1062 부산광역시 중구 중앙대로2 (중앙동7가)
# 1206 대전광역시 동구 용운동423-22
# 1377 대전광역시 동구 홍도동92-3
# 1568 경상북도 구미시 오태동659-13
# 1647 강원도 태백시 소도동326
# 1811 경상남도 창원시 진해구 명동580 좌측 주차장
# 2397 울산광역시 남구 선암동584-8번지

In [117]:
idx5 = [16, 163, 1062, 1206, 1377, 1568, 1647, 1811, 2397]
adr[idx5]

array(['제주특별자치도 제주시 연동2334-4', '경상남도 창원시 성산구 사파동64-2',
       '부산광역시 중구 중앙대로2 (중앙동7가)', '대전광역시 동구 용운동423-22', '대전광역시 동구 홍도동92-3',
       '경상북도 구미시 오태동659-13', '강원도 태백시 소도동326',
       '경상남도 창원시 진해구 명동580 좌측 주차장', '울산광역시 남구 선암동584-8번지'], dtype=object)

In [118]:
s = '제주특별자치도 제주시 연동2334-4'
split8 = re.compile('([가-힣]+)([0-9]+)')
split8.sub('\g<1> \g<2>', s, 1)

'제주특별자치도 제주시 연동 2334-4'

In [119]:
s = '부산광역시 중구 중앙대로2 (중앙동7가)'
split8.sub('\g<1> \g<2>', s, 1)

'부산광역시 중구 중앙대로 2 (중앙동7가)'

In [120]:
s = '경상남도 창원시 진해구 명동580 좌측 주차장'
split8.sub('\g<1> \g<2>', s, 1)

'경상남도 창원시 진해구 명동 580 좌측 주차장'

In [121]:
for i in idx5:
    temp = adr[i]
    adr[i] = split8.sub('\g<1> \g<2>', temp, 1)

In [122]:
adr[idx5]

array(['제주특별자치도 제주시 연동 2334-4', '경상남도 창원시 성산구 사파동 64-2',
       '부산광역시 중구 중앙대로 2 (중앙동7가)', '대전광역시 동구 용운동 423-22',
       '대전광역시 동구 홍도동 92-3', '경상북도 구미시 오태동 659-13', '강원도 태백시 소도동 326',
       '경상남도 창원시 진해구 명동 580 좌측 주차장', '울산광역시 남구 선암동 584-8번지'], dtype=object)

In [124]:
split9 = re.compile('[가-힣]+[0-9]*[가-힣]+[0-9]+')
count = 0
for i, one in enumerate(adr):
    if split9.search(one):
        count += 1
        print(i, one)

1 광주광역시 북구 첨단벤처로108번길 9
10 전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소)
15 전라북도 전주시 완산구 신촌3길 24번지
35 대전광역시 유성구 반석로12번길 18-4
43 서울특별시 마포구 마포대로1길 9
44 서울특별시 성동구 광나루로4가길 23
77 제주특별자치도 제주시 외도1동 482-1
78 제주특별자치도 제주시 이도1동 1293-1
79 제주특별자치도 제주시 중앙로25길 17
80 제주특별자치도 제주시 일도2동 488-1
82 제주특별자치도 제주시 하귀1리 176-3
88 경기도 용인시 처인구 주북로94번길 30-1 (고림동)
98 경기도 용인시 기흥구 중부대로 375 지하2층 10
99 경기도 수원시 권선구 동수원로 232 지하2층 주차장 좌측
100 경기도 시흥시 서울대학로278번길 67 지상4층 주차장 A4
111 강원도 평창군 대화면 남산1길 28
116 경기도 가평군 설악면 미사리로540번길 51
120 충청북도 제천시 의림대로42길 34
131 경기도 화성시 동탄공원로2길 32
138 제주특별자치도 서귀포시 소보리당로164번길 62  주차장 좌측
165 경상남도 양산시 삼호1길 34 4층 주차장 02
166 경상남도 진주시 동진로 440 지하2층 주차장 C6
171 울산광역시 동구 진성8길 119  입구 소형차 주차구역
177 대구광역시 동구 아양로37길 92
178 대구광역시 북구 복현로8길 16-5
180 경상북도 칠곡군 동명면 동명팔거천2길 9 복지센터 앞
197 제주특별자치도 제주시 봉개북1길 15 봉개동주민센터
212 전라북도 정읍시 북면 하북1길 227-17 (남산리)
215 경기도 이천시 마장면 중부대로609번길 244-35 (목리)
216 경기도 이천시 신둔면 서이천로853번길 346-129 (용면리)
220 제주특별자치도 제주시 용담2동 2002
222 전라남도 여수시 국포1로 36 (국동)
232 전라남도 곡성군 곡성읍 학교로8
264 경상남도 함안군 칠원읍 용산1길 28
285 전라북

In [125]:
count

577

In [126]:
# 부착

# 1392 서울특별시 마포구 상암동 월드컵북로 58길 15 지하4층 18

In [127]:
adr[1392]

'서울특별시 마포구 상암동 월드컵북로 58길 15 지하4층 18'

In [128]:
adr[1392] = '서울특별시 마포구 상암동 월드컵북로58길 15 지하4층 18'
adr[1392]

'서울특별시 마포구 상암동 월드컵북로58길 15 지하4층 18'

In [ ]:
# 분리 주의

# 2994 경상남도 창녕군 유어면 우포1대로583

In [129]:
adr[2994]

'경상남도 창녕군 유어면 우포1대로583'

In [130]:
adr[2994] = '경상남도 창녕군 유어면 우포1대로 583'
adr[2994]

'경상남도 창녕군 유어면 우포1대로 583'

In [132]:
# 분리

# 232 전라남도 곡성군 곡성읍 학교로8
# 285 전라북도 익산시 함열읍 와리164
# 1195 경상북도 영천시 금호읍 교대리74
# 1978 경기도 파주시 적성면 설마리48-3
# 2329 경기도 군포시 산본로267
# 2444 충청남도 부여군 홍산면 북촌리188-7
# 2587 경상북도 의성군 비안면 강변길59
# 2628 경상북도 경주시 산내면 문복로1822
# 2852 강원도 태백시 보드미길8(장성동)
# 2961 경상남도 창녕군 창녕읍 탐하로201
# 2963 충청남도 보령시 대천로113
# 2966 충청북도 음성군 맹동면 원중로1339
# 2968 경상남도 의령군 구룡16
# 2970 경상북도 구미시 도개면 도안로103
# 2972 경상남도 의령군 유곡면 청정로1679
# 2977 울산광역시 울주군 삼동면 보삼길550
# 2979 충청북도 영동군 용산면 영동산단로27
# 2981 경상북도 문경시 중앙로50
# 2985 충청북도 영동군 매곡면 민주지산로3595
# 2989 전라남도 영암군 삼호읍 공항로21
# 2991 강원도 강릉시 성산면 삼포암길133
# 2997 경상북도 청송군 주왕산면 부동로1008
# 3012 전라남도 장흥군 용산면 장흥대로2667
# 3014 경상남도 거창군 신원면 신차로2924
# 3022 경상북도 구미시 경부고속도로172
# 3031 경상북도 봉화군 봉성면 부랭이길88
# 3035 경상북도 청송군 진보면 청송로6359
# 3045 충청남도 청양군 비봉면 충절로1994
# 3050 충청남도 서산시 문화로112
# 3054 경상북도 봉화군 상운면 예봉로1281
# 3057 충청남도 부여군 은산면 충의로694
# 3062 경상북도 영양군 영양읍 군청길37
# 3068 충청남도 부여군 세도면 세도중앙로8
# 3071 경상북도 영주시 풍기읍 소백로2156
# 3075 인천광역시 남동구 남동서로209
# 3081 경상북도 청송군 청송읍 주왕산로177
# 3086 경상남도 의령군 부림면 입산로178
# 3091 경상북도 청송군 청송읍 군청로25
# 3094 대전광역시 유성구 신성로55
# 3097 경상북도 구미시 형곡로180
# 3099 경상남도 의령군 낙서면 낙동강로859
# 3101 경상북도 안동시 관광단지로316
# 3133 충청남도 예산군 삽교읍 예학로65
# 3170 경상북도 청송군 안덕면 방호정로50
# 3175 경상북도 청송군 현서면 달정길9
# 3178 경상북도 청송군 주왕산면 이전앞길27
# 3188 광주광역시 북구 하서로111
# 3195 경상북도 청송군 청송읍 중앙로184
# 3200 대전광역시 유성구 유성대로1719
# 3212 충청북도 영동군 용화면 휴양림길60
# 3217 경기도 화성시 송산면 평택시흥고속도로25
# 3241 전라북도 정읍시 내장호반로266
# 3253 전라북도 익산시 왕궁면 궁성로666
# 3263 전라북도 고창군 해리면 하련리189-5
# 3274 전라북도 군산시 창길21
# 3281 강원도 정선군 사북읍 사북중앙로65
# 3298 경기도 구리시 장자대로141
# 3366 경기도 용인시 수지구 경부고속도로400
# 3374 경상북도 의성군 사곡면 의성사곡로966
# 3400 전라남도 보성군 회천면 우암길21
# 3402 충청남도 금산군 금성면 의총길25
# 3410 전라북도 남원시 이백로13
# 3414 경상북도 구미시 광평길55
# 3423 경상북도 영주시 영주로204
# 3428 경상북도 문경시 중앙로280
# 3433 경상북도 구미시 해마루공원로24
# 3437 경상북도 예천군 효자면 은풍로1154
# 3440 경상북도 울진군 온정면 백암온천로1185
# 3471 광주광역시 서구 천변우하로391
# 3519 경상남도 창원시 마산합포구 진동면 신서촌로15
# 3542 경상남도 함안군 가야읍 말산리113
# 3552 경상남도 의령군 지정면 봉곡리511-2
# 3668 전라북도 남원시 운봉읍 용산리266
# 3670 충청북도 청주시 서원구 흥덕로48
# 3695 충청남도 아산시 실옥로234
# 3701 광주광역시 북구 추암로249
# 3727 경상남도 산청군 산청읍 물안실로8
# 3888 울산광역시 남구 갈밭로49

#### [분리] -[문자] [숫자]

In [131]:
idx6 = [232, 285, 1195, 1978, 2329, 2444, 2587, 2628, 2852, 2961, 2963, 2966, 2968, 2972, 2977, 2979, 2981, 2985, 2989, 2991, 2997, 3012, 3014, 3022, 3031, 3035, 3045, 3050, 3054, 3057, 3062,
        3068, 3071, 3075, 3081, 3086, 3091, 3094, 3097, 3099, 3101, 3133, 3170, 3175, 3178, 3188, 3195, 3200, 3212, 3217, 3241, 3253, 3263, 3274, 3281, 3298, 3366, 3374, 3400, 3402, 3410, 3414, 
        3423, 3428, 3433, 3437, 3440, 3471, 3519, 3542, 3552, 3668, 3670, 3695, 3701, 3727, 3888]

In [133]:
split10 = re.compile('([가-힣]+)([0-9]+)')
s = '경상남도 의령군 지정면 봉곡리511-2'
split10.sub('\g<1> \g<2>', s, 1)

'경상남도 의령군 지정면 봉곡리 511-2'

In [134]:
s = '광주광역시 북구 추암로249'
split10.sub('\g<1> \g<2>', s, 1)

'광주광역시 북구 추암로 249'

In [135]:
for i in idx6:
    temp = adr[i]
    adr[i] = split10.sub('\g<1> \g<2>', temp, 1)

In [136]:
adr[idx6]

array(['전라남도 곡성군 곡성읍 학교로 8', '전라북도 익산시 함열읍 와리 164', '경상북도 영천시 금호읍 교대리 74',
       '경기도 파주시 적성면 설마리 48-3', '경기도 군포시 산본로 267',
       '충청남도 부여군 홍산면 북촌리 188-7', '경상북도 의성군 비안면 강변길 59',
       '경상북도 경주시 산내면 문복로 1822', '강원도 태백시 보드미길 8(장성동)',
       '경상남도 창녕군 창녕읍 탐하로 201', '충청남도 보령시 대천로 113',
       '충청북도 음성군 맹동면 원중로 1339', '경상남도 의령군 구룡 16', '경상남도 의령군 유곡면 청정로 1679',
       '울산광역시 울주군 삼동면 보삼길 550', '충청북도 영동군 용산면 영동산단로 27',
       '경상북도 문경시 중앙로 50', '충청북도 영동군 매곡면 민주지산로 3595',
       '전라남도 영암군 삼호읍 공항로 21', '강원도 강릉시 성산면 삼포암길 133',
       '경상북도 청송군 주왕산면 부동로 1008', '전라남도 장흥군 용산면 장흥대로 2667',
       '경상남도 거창군 신원면 신차로 2924', '경상북도 구미시 경부고속도로 172',
       '경상북도 봉화군 봉성면 부랭이길 88', '경상북도 청송군 진보면 청송로 6359',
       '충청남도 청양군 비봉면 충절로 1994', '충청남도 서산시 문화로 112',
       '경상북도 봉화군 상운면 예봉로 1281', '충청남도 부여군 은산면 충의로 694',
       '경상북도 영양군 영양읍 군청길 37', '충청남도 부여군 세도면 세도중앙로 8',
       '경상북도 영주시 풍기읍 소백로 2156', '인천광역시 남동구 남동서로 209',
       '경상북도 청송군 청송읍 주왕산로 177', '경상남도 의령군 부림면 입산로 178',
       '경상북도 청송군 청송읍 군청로 25', '대

#### [부착] -로 -번길|길

In [138]:
split11 = re.compile('([가-힣]+[0-9]*로) ([0-9]+[가-힣]+)')
idx7 = []
count = 0
for i, one in enumerate(adr):
    if split11.search(one):
        idx7.append(i)
        count += 1
        print(i, one)

27 대전광역시 대덕구 석봉로 38번길 55 대덕구보건소
94 광주광역시 서구 경열로 17번길 5
145 제주특별자치도 서귀포시 성산읍 일주동로 4120번길 7 동부소방서
189 충청남도 논산시 연무로 166번길 64 2주차장
196 제주특별자치도 제주시 선덕로 8길 12 설문대여성문화센터
236 대전광역시 중구 동서대로 1322번길 81
311 경기도 가평군 설악면 미사리로 540번길 51 가평휴게소  SK 가평주유소(춘천방향) 옆
362 광주광역시 광산구 광산로 68번길 13
365 제주특별자치도 서귀포시 동홍중앙로 7번길 19 자기주도학습지원센터
406 충청남도 홍성군 홍북읍 홍북로 453번길 50
433 대구광역시 달성군 구지면 창리로 11길 38
444 제주특별자치도 서귀포시 남성중로 153번길 15
446 제주특별자치도 서귀포시 효돈로 152번길 7
480 전라남도 여수시 학동 168(시청로 1번지)
535 서울특별시 영등포구 여의대방로 53길 22
537 서울특별시 강북구 도봉로 69길 29
578 서울특별시 서초구 남부순환로 340길 58
646 경상북도 구미시 봉곡로 15길 54
675 부산광역시 중구 대청로 116번길 3
692 경상남도 김해시 번화1로 56번길 15 롯데마트
724 인천광역시 중구 공항동로 296번길 69
739 강원도 철원군 철원읍 금학로 330번길 12
827 전라남도 화순군 능주면 학포로 2275번지
867 경기도 고양시 덕양구 화중로 104번길 13 입구 좌측
888 경기도 포천시 소홀읍 호국로 56번지
997 제주특별자치도 서귀포시 대정읍 추사로 36번길 1
1049 서울특별시 중구 소공로 51번지
1091 경기도 고양시 일산동구 중앙로 1275번길 64
1094 서울특별시 성동구 고산자로 26길 6
1111 경기도 이천시 마장면 덕이로 154번길 287-76
1116 경기도 성남시 분당구 야탑로 111번길 12-14
1117 경기도 성남시 분당구 성남대로 43번길 11
1214 충청북도 제천시 용두대로 11길 10

In [139]:
count

89

In [ ]:
# 827 전라남도 화순군 능주면 학포로 2275번지
# 888 경기도 포천시 소홀읍 호국로 56번지
# 1049 서울특별시 중구 소공로 51번지
# 1765 경상북도 영천시 한방로 53번지
# 2111 전라북도 전주시 완산구 서곡로 100번지

In [ ]:
idx7.remove(827)

In [142]:
idx7.remove(888)
idx7.remove(1049)
idx7.remove(1765)

ValueError: list.remove(x): x not in list

In [143]:
idx7.remove(2111)

In [144]:
s = '대전광역시 대덕구 석봉로 38번길 55 대덕구보건소'
# split11.search(s)
split11.sub('\g<1>\g<2>', s, 1)

'대전광역시 대덕구 석봉로38번길 55 대덕구보건소'

In [145]:
s = '전라남도 여수시 학동 168(시청로 1번지)'
# split11.search(s)
split11.sub('\g<1>\g<2>', s, 1)

'전라남도 여수시 학동 168(시청로1번지)'

In [146]:
for i in idx7:
    temp = adr[i]
    adr[i] = split11.sub('\g<1>\g<2>', temp, 1)

In [147]:
adr[idx7]

array(['대전광역시 대덕구 석봉로38번길 55 대덕구보건소', '광주광역시 서구 경열로17번길 5',
       '제주특별자치도 서귀포시 성산읍 일주동로4120번길 7 동부소방서', '충청남도 논산시 연무로166번길 64 2주차장',
       '제주특별자치도 제주시 선덕로8길 12 설문대여성문화센터', '대전광역시 중구 동서대로1322번길 81',
       '경기도 가평군 설악면 미사리로540번길 51 가평휴게소  SK 가평주유소(춘천방향) 옆',
       '광주광역시 광산구 광산로68번길 13', '제주특별자치도 서귀포시 동홍중앙로7번길 19 자기주도학습지원센터',
       '충청남도 홍성군 홍북읍 홍북로453번길 50', '대구광역시 달성군 구지면 창리로11길 38',
       '제주특별자치도 서귀포시 남성중로153번길 15', '제주특별자치도 서귀포시 효돈로152번길 7',
       '전라남도 여수시 학동 168(시청로1번지)', '서울특별시 영등포구 여의대방로53길 22',
       '서울특별시 강북구 도봉로69길 29', '서울특별시 서초구 남부순환로340길 58',
       '경상북도 구미시 봉곡로15길 54', '부산광역시 중구 대청로116번길 3',
       '경상남도 김해시 번화1로56번길 15 롯데마트', '인천광역시 중구 공항동로296번길 69',
       '강원도 철원군 철원읍 금학로330번길 12', '경기도 고양시 덕양구 화중로104번길 13 입구 좌측',
       '제주특별자치도 서귀포시 대정읍 추사로36번길 1', '경기도 고양시 일산동구 중앙로1275번길 64',
       '서울특별시 성동구 고산자로26길 6', '경기도 이천시 마장면 덕이로154번길 287-76',
       '경기도 성남시 분당구 야탑로111번길 12-14', '경기도 성남시 분당구 성남대로43번길 11',
       '충청북도 제천시 용두대로11길 10', '서울특별시 양천구 남부순환로88길 5-7',
 

#### [부착] -길-길

In [148]:
split12 = re.compile('([가-힣]+길) ([가-힣0-9]+길)')
for i, one in enumerate(adr):
    if split12.search(one):
        print(i, one)

2739 경상남도 진주시 망경남길 60번길 18


In [149]:
adr[2739]

'경상남도 진주시 망경남길 60번길 18'

In [150]:
adr[2739] = '경상남도 진주시 망경남길60번길 18'
adr[2739]

'경상남도 진주시 망경남길60번길 18'

#### [부착] -동-[숫자]가

In [152]:
split13 = re.compile('([가-힣]+동) ([0-9]가)')
for i, one in enumerate(adr):
    if split13.search(one):
        print(i, one)

283 전라북도 전주시 덕진구 덕진동 2가 179-6
607 전라북도 전주시 덕진동 2가 694-1


In [153]:
adr[[283, 607]]

array(['전라북도 전주시 덕진구 덕진동 2가 179-6', '전라북도 전주시 덕진동 2가 694-1'], dtype=object)

In [154]:
for i, one in enumerate(adr):
    if split13.search(one):
        adr[i] = split13.sub('\g<1>\g<2>', one, 1)

In [155]:
adr[[283, 607]]

array(['전라북도 전주시 덕진구 덕진동2가 179-6', '전라북도 전주시 덕진동2가 694-1'], dtype=object)

### 주소 세분류

#### 빈칸 처리

In [156]:
count = 0
for i, one in enumerate(adr):
    if '  ' in one:
        adr[i] = one.replace('  ', ' ')
        count += 1
count

205

#### 컴마 제거

In [157]:
count = 0
for i, one in enumerate(adr):
    if ',' in one:
        adr[i] = one.replace(',', '')
        count += 1
count

0

#### 시-구 같은 구분지역 처리

In [158]:
split_dict = {' 고양시 덕양구 ':'$고양시-덕양구$', ' 고양시 일산동구 ':'$고양시-일산동구$', ' 고양시 일산서구 ':'$고양시-일산서구$', ' 성남시 분당구 ':'$성남시-분당구$', ' 성남시 수정구 ':'$성남시-수정구$', 
              ' 성남시 중원구 ':'$성남시-중원구$', ' 성남시 권선구 ':'$성남시-권선구$', ' 성남시 영통구 ':'$성남시-영통구$', ' 성남시 장안구 ':'$성남시-장안구$', ' 성남시 팔달구 ':'$성남시-팔달구$',
              ' 수원시 권선구 ':'$수원시-권선구$', ' 수원시 영통구 ':'$수원시-영통구$', ' 수원시 장안구 ':'$수원시-장안구$', ' 수원시 팔달구 ':'$수원시-팔달구$', ' 안산시 단원구 ':'$안산시-단원구$', 
              ' 안산시 상록구 ':'$안산시-상록구$', ' 안양시 동안구 ':'$안양시-동안구$', ' 안양시 만안구 ':'$안양시-만안구$', ' 용인시 기흥구 ':'$용인시-기흥구$', ' 용인시 수지구 ':'$용인시-수지구$', 
              ' 용인시 처인구 ':'$용인시-처인구$', ' 청주시 상당구 ':'$청주시-상당구$', ' 청주시 서원구 ':'$청주시-서원구$', ' 청주시 청원구 ':'$청주시-청원구$', ' 청주시 흥덕구 ':'$청주시-흥덕구$', 
              ' 천안시 동남구 ':'$천안시-동남구$', ' 천안시 서북구 ':'$천안시-서북구$', ' 전주시 덕진구 ':'$전주시-덕진구$', ' 전주시 완산구 ':'$전주시-완산구$', ' 창원시 마산합포구 ':'$창원시-마산합포구$', 
              ' 창원시 마산회원구 ':'$창원시-마산회원구$', ' 창원시 성산구 ':'$창원시-성산구$', ' 창원시 의창구 ':'$창원시-의창구$', ' 창원시 진해구 ':'$창원시-진해구$', ' 포항시 남구 ':'$포항시-남구$', 
              ' 포항시 북구 ':'$포항시-북구$'}

In [159]:
len(split_dict)

36

In [160]:
adr2 = []
for ad in tqdm(adr):
    for k, v in split_dict.items():
        ad = ad.replace(k, v)
    ad = ad.replace(' ', '$')
    ad = ad.replace('-', ' ')
    adr2.append(ad.split('$'))

100%|██████████| 4365/4365 [00:00<00:00, 67322.20it/s]


In [161]:
adr2[:5]

[['강원도', '인제군', '상남면', '하남리', '339 7(양양방향)'],
 ['광주광역시', '북구', '첨단벤처로108번길', '9'],
 ['전라남도', '순천시', '해룡면', '남가리', '733 73'],
 ['전라남도', '구례군', '산동면', '산업로', '2319'],
 ['전라남도', '구례군', '산동면', '노고단로', '1068']]

In [228]:
mod_adr2 = []
for one in tqdm(adr2):
    temp = []
    for two in one:
        if two not in temp:
            temp.append(two)
    mod_adr2.append(temp)

100%|██████████| 4365/4365 [00:00<00:00, 436375.57it/s]


#### 지역 구분으로 주소 분리
- 지역명 구분
    - 지역
    - 시군구
    - 읍면동
    - 리
    - 도로명

In [229]:
ad1_p = re.compile('[가-힣]+도$|[가-힣]+시$')
ad2_p = re.compile('[가-힣]+[시군구]$')
ad3_p = re.compile('[가-힣0-9]+[읍면동가]$')
ad4_p = re.compile('[가-힣0-9]+리$')
ad5_p = re.compile('[가-힣0-9]+[로길번길]$')

In [230]:
s = '세종특별자치시'
ad1_p.search(s)

<re.Match object; span=(0, 7), match='세종특별자치시'>

In [231]:
s = '제주특별자치도'
ad1_p.search(s)

<re.Match object; span=(0, 7), match='제주특별자치도'>

In [232]:
s = '고양시 덕양구'
ad2_p.search(s)

<re.Match object; span=(4, 7), match='덕양구'>

In [233]:
s = '강남구'
ad2_p.search(s)

<re.Match object; span=(0, 3), match='강남구'>

In [234]:
s = '효자동3가'
ad3_p.search(s)

<re.Match object; span=(0, 5), match='효자동3가'>

In [235]:
s = '서귀동'
ad3_p.search(s)

<re.Match object; span=(0, 3), match='서귀동'>

In [236]:
s = '신길1동'
ad3_p.search(s)

<re.Match object; span=(0, 4), match='신길1동'>

In [237]:
s = '죽림리'
ad4_p.search(s)

<re.Match object; span=(0, 3), match='죽림리'>

In [238]:
s = '첨단벤처로108번길'
ad5_p.search(s)

<re.Match object; span=(0, 10), match='첨단벤처로108번길'>

In [239]:
s = '노형9길'
ad5_p.search(s)

<re.Match object; span=(0, 4), match='노형9길'>

In [240]:
for i, adrs in tqdm(enumerate(mod_adr2)):
    for j, one in enumerate(adrs):
        if j == 0:
            if ad1_p.search(one):
                pass
            else:
                print(i, j, mod_adr2[i])
        elif j == 1:
            if ad2_p.search(one):
                pass
            else:
                adrs.insert(j, np.nan)
                print(i, j, mod_adr2[i])
        elif j == 2:
            if ad3_p.search(one):
                pass
            else:
                adrs.insert(j, np.nan)
                print(i, j, mod_adr2[i])
        elif j == 3:
            if ad4_p.search(one):
                pass
            else:
                adrs.insert(j, np.nan)
                print(i, j, mod_adr2[i])


4365it [00:00, 42550.35it/s]

1 2 ['광주광역시', '북구', nan, '첨단벤처로108번길', '9']
1 3 ['광주광역시', '북구', nan, nan, '첨단벤처로108번길', '9']
3 3 ['전라남도', '구례군', '산동면', nan, '산업로', '2319']
4 3 ['전라남도', '구례군', '산동면', nan, '노고단로', '1068']
9 3 ['전라남도', '무안군', '망운면', nan, '공항로', '970 260']
10 3 ['전라남도', '무안군', '삼향읍', nan, '오룡1길', '(E주차장 전라남도관광안내소)']
12 3 ['전라북도', '정읍시', '내장동', nan, '52']
13 3 ['전라북도', '정읍시', '내장동', nan, '169 1']
15 2 ['전라북도', '전주시 완산구', nan, '신촌3길', '24번지']
15 3 ['전라북도', '전주시 완산구', nan, nan, '신촌3길', '24번지']
16 3 ['제주특별자치도', '제주시', '연동', nan, '2334 4']
18 3 ['제주특별자치도', '서귀포시', '성산읍', nan, '해맞이해안로', '2660']
19 3 ['제주특별자치도', '제주시', '애월읍', nan, '애월해안로', '400 9']
20 2 ['제주특별자치도', '제주시', nan, '연북로', '1(노형동)']
20 3 ['제주특별자치도', '제주시', nan, nan, '연북로', '1(노형동)']
21 1 ['세종특별자치시', nan, '연기면', '정안세종로', '1527', '은하수공원', '화장시설', '주차장']
21 3 ['세종특별자치시', nan, '연기면', nan, '정안세종로', '1527', '은하수공원', '화장시설', '주차장']
22 2 ['서울특별시', '영등포구', nan, '영중로', '125', '지상주차장(외부', '1F)']
22 3 ['서울특별시', '영등포구', nan, nan, '영중로', '125', '지상주차장(외부', '1F)']


In [241]:
# 5번째 구분에 '-리', 주소 존재 여부 체크
for i, one in enumerate(mod_adr2):
    try:
        if ad4_p.search(one[4]):
            print(i, one)
    except:
        pass

In [242]:
# 6번째 구분에 '-리', '-로, -길' 주소 존재 여부 체크
for i, one in enumerate(mod_adr2):
    try:
        if ad4_p.search(one[5]) or ad5_p.search(one[5]):
            print(i, one)
    except:
        pass

169 ['경상남도', '통영시', nan, nan, '큰발개', '1길', '33', '마리나요트클럽', '앞']
405 ['충청남도', '천안시 서북구', nan, nan, '신당새터', '2길', '28', '카페', '옆']
509 ['경기도', '양평군', '양동면', '단석리', '513 2', '(여양3로', '384 27)']
901 ['인천광역시', '서구', nan, nan, '청라', '커낼로', '252(경서동)']
1034 ['경기도', '하남시', nan, nan, '미사강변', '한강로', '299', '지하주차장', '입구', '앞']
1041 ['대전광역시', '유성구', nan, nan, '테크노', '9로', '35', '입구', '국기계양대', '왼쪽']
1357 ['제주특별자치도', '제주시', nan, nan, '산천단', '동3길', '2', '북쪽주차장', '입구']
1825 ['경기도', '부천시', nan, nan, '원미구', '석천로', '188', '지상주차장', '2층', 'D구역', '창고', '오른쪽']
1918 ['경상북도', '상주시', nan, nan, '영남', '제일로', '1339']
1921 ['세종특별자치시', nan, nan, nan, '다솜', '3로', '66']
2090 ['충청남도', '당진시', '정미면', '천의리', '245 4', '천의삼거리', '주차장', '내']
2215 ['경기도', '이천시', '호법면', nan, '프리미엄', '아울렛로', '177 74']
2498 ['충청남도', '보령시', nan, nan, '갓바위', '1길', '10 30']
2736 ['울산광역시', '북구', nan, nan, '진장', '유통로', '64']
2971 ['경상남도', '합천군', '합천읍', nan, '교동', '1길', '33']
2980 ['충청북도', '괴산군', '칠성면', nan, '명태재로외사', '4길', '1']
2995 ['경상남도', '통영시', '

In [243]:
# 169, 405, 901, 1034, 1041, 1357, 1825, 1918, 1921, 2090, 2215, 2498, 2736, 2971, 2980, 2995, 3040, 3080, 3100, 3409, 3422, 3427, 3439, 3761

In [244]:
len([169, 405, 901, 1034, 1041, 1357, 1825, 1918, 1921, 2090, 2215, 2498, 2736, 2971, 2980, 2995, 3040, 3080, 3100, 3409, 3422, 3427, 3439, 3761])

24

#### 하나씩 확인

In [245]:
idx8 = [169, 405, 901, 1034, 1041, 1357, 1825, 1918, 1921, 2090, 2215, 2498, 2736, 2971, 2980, 2995, 3040, 3080, 3100, 3409, 3422, 3427, 3439, 3761]

In [246]:
for i in idx8:
    mod_adr2[i][4] = mod_adr2[i][4] + mod_adr2[i][5]
    mod_adr2[i].pop(5)
    print(mod_adr2[i])

['경상남도', '통영시', nan, nan, '큰발개1길', '33', '마리나요트클럽', '앞']
['충청남도', '천안시 서북구', nan, nan, '신당새터2길', '28', '카페', '옆']
['인천광역시', '서구', nan, nan, '청라커낼로', '252(경서동)']
['경기도', '하남시', nan, nan, '미사강변한강로', '299', '지하주차장', '입구', '앞']
['대전광역시', '유성구', nan, nan, '테크노9로', '35', '입구', '국기계양대', '왼쪽']
['제주특별자치도', '제주시', nan, nan, '산천단동3길', '2', '북쪽주차장', '입구']
['경기도', '부천시', nan, nan, '원미구석천로', '188', '지상주차장', '2층', 'D구역', '창고', '오른쪽']
['경상북도', '상주시', nan, nan, '영남제일로', '1339']
['세종특별자치시', nan, nan, nan, '다솜3로', '66']
['충청남도', '당진시', '정미면', '천의리', '245 4천의삼거리', '주차장', '내']
['경기도', '이천시', '호법면', nan, '프리미엄아울렛로', '177 74']
['충청남도', '보령시', nan, nan, '갓바위1길', '10 30']
['울산광역시', '북구', nan, nan, '진장유통로', '64']
['경상남도', '합천군', '합천읍', nan, '교동1길', '33']
['충청북도', '괴산군', '칠성면', nan, '명태재로외사4길', '1']
['경상남도', '통영시', '광도면', nan, '죽림2로', '45']
['충청남도', '천안시 동남구', '병천면', nan, '병천2로', '57']
['충청남도', '논산시', nan, nan, '산양1길', '2']
['경기도', '파주시', nan, nan, '하우3길', '77']
['경상북도', '구미시', '산동면', nan, '첨단기업1로', '17']
['전라북도

In [247]:
mod_adr2[2090]

['충청남도', '당진시', '정미면', '천의리', '245 4천의삼거리', '주차장', '내']

In [248]:
mod_adr2[2090][4] = '천의삼거리'
mod_adr2[2090].insert(4, '4')
mod_adr2[2090].insert(4, '245')
mod_adr2[2090]

['충청남도', '당진시', '정미면', '천의리', '245', '4', '천의삼거리', '주차장', '내']

In [249]:
mod_adr2[4362]

['전라남도', '장흥군', '부산면', '호계리', '산215 2']

In [252]:
# 5번째 구분에 '-로, -길' 주소 존재 여부 체크
count = 0
idx9 = []
for i, one in enumerate(mod_adr2):
    try:
        if not ad5_p.search(one[4]):
            idx9.append(i)
            count += 1
            print(i, one)
    except:
        pass

0 ['강원도', '인제군', '상남면', '하남리', '339 7(양양방향)']
2 ['전라남도', '순천시', '해룡면', '남가리', '733 73']
5 ['전라남도', '장성군', '북하면', '약수리', '252 6']
6 ['전라남도', '장성군', '북하면', '신성리', '427 2']
7 ['전라북도', '부안군', '변산면', '격포리', '283 1']
8 ['전라남도', '영암군', '영암읍', '개신리', '484 47']
11 ['전라북도', '남원시', '산내면', '덕동리', '406']
12 ['전라북도', '정읍시', '내장동', nan, '52']
13 ['전라북도', '정읍시', '내장동', nan, '169 1']
14 ['전라북도', '무주군', '설천면', '삼공리', '411']
16 ['제주특별자치도', '제주시', '연동', nan, '2334 4']
17 ['제주특별자치도', '제주시', '조천읍', '함덕리', '906번지']
28 ['대전광역시', '대덕구', '신대동', nan, '169 1']
30 ['대전광역시', '서구', '만년동', nan, '74']
31 ['대전광역시', '서구', '복수동', nan, '613']
32 ['대전광역시', '서구', '월평동', nan, '1275']
33 ['대전광역시', '서구', '정림동', nan, '127 1']
34 ['대전광역시', '중구', '사정동', nan, '403번지']
36 ['경기도', '양주시', '옥정동', nan, '1039']
39 ['경기도', '부천시', '상동', nan, '549번지']
41 ['경기도', '안산시 상록구', '이동', nan, '620']
42 ['경기도', '하남시', '망월동', nan, '1 1']
45 ['인천광역시', '서구', '연희동', nan, '187 8']
46 ['인천광역시', '남동구', '논현동', nan, '762 3']
47 ['인천광역시', '남동구', '서창동', nan, '

In [253]:
count

1536

In [254]:
for i in tqdm(idx9):
    mod_adr2[i].insert(4, np.nan)

100%|██████████| 1536/1536 [00:00<00:00, 1539782.73it/s]


In [257]:
idx9[:5]

[0, 2, 5, 6, 7]

In [259]:
mod_adr2[0]

['강원도', '인제군', '상남면', '하남리', nan, '339 7(양양방향)']

#### df 만들기

In [260]:
sliced_adr2 = [one[:5] for one in mod_adr2]

In [261]:
sliced_adr2_df = pd.DataFrame(sliced_adr2, columns=['지역', '시군구', '읍면동', '리', '도로명'])
sliced_adr2_df.head()

,지역,시군구,읍면동,리,도로명
0,강원도,인제군,상남면,하남리,NaN
1,광주광역시,북구,NaN,NaN,첨단벤처로108번길
2,전라남도,순천시,해룡면,남가리,NaN
3,전라남도,구례군,산동면,NaN,산업로
4,전라남도,구례군,산동면,NaN,노고단로


In [262]:
sliced_adr2_df.shape

(4365, 5)

In [269]:
for one in sliced_adr2_df['도로명'].unique():
    if not (('로' in str(one)) or ('길' in str(one)) or ('번길' in str(one))) and (one != np.nan):
        print(one)

nan
None


In [270]:
len(df2['주소'])

4365

In [271]:
sliced_adr2_df['주소'] = df2['주소']
sliced_adr2_df.head()

,지역,시군구,읍면동,리,도로명,주소
0,강원도,인제군,상남면,하남리,NaN,강원도 인제군 상남면 하남리 339-7(양양방향)
1,광주광역시,북구,NaN,NaN,첨단벤처로108번길,광주광역시 북구 첨단벤처로108번길 9
2,전라남도,순천시,해룡면,남가리,NaN,전라남도 순천시 해룡면 남가리 733-73
3,전라남도,구례군,산동면,NaN,산업로,전라남도 구례군 산동면 산업로 2319
4,전라남도,구례군,산동면,NaN,노고단로,전라남도 구례군 산동면 노고단로 1068


In [272]:
sliced_adr2_df.tail()

,지역,시군구,읍면동,리,도로명,주소
4360,전라북도,장수군,장수읍,NaN,장천로,전라북도 장수군 장수읍 장천로 247
4361,인천광역시,서구,NaN,NaN,정서진1로,인천광역시 서구 정서진1로 41
4362,전라남도,장흥군,부산면,호계리,NaN,전라남도 장흥군 부산면 호계리 산215-2
4363,제주특별자치도,제주시,NaN,NaN,천수로,제주특별자치도 제주시 천수로 72
4364,경기도,용인시 처인구,NaN,NaN,동백죽전대로,경기도 용인시 처인구 동백죽전대로 61


In [273]:
sliced_adr2_df2 = sliced_adr2_df.copy()
sliced_adr2_df2.columns = ['ad1', 'ad2', 'ad3', 'ad4', 'ad5', '주소']
sliced_adr2_df2 = sliced_adr2_df2[['주소', 'ad1', 'ad2', 'ad3', 'ad4', 'ad5']]
sliced_adr2_df2.head()

,주소,ad1,ad2,ad3,ad4,ad5
0,강원도 인제군 상남면 하남리 339-7(양양방향),강원도,인제군,상남면,하남리,NaN
1,광주광역시 북구 첨단벤처로108번길 9,광주광역시,북구,NaN,NaN,첨단벤처로108번길
2,전라남도 순천시 해룡면 남가리 733-73,전라남도,순천시,해룡면,남가리,NaN
3,전라남도 구례군 산동면 산업로 2319,전라남도,구례군,산동면,NaN,산업로
4,전라남도 구례군 산동면 노고단로 1068,전라남도,구례군,산동면,NaN,노고단로


In [276]:
sliced_adr2_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4365 entries, 0 to 4364
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   주소      4365 non-null   object
 1   ad1     4365 non-null   object
 2   ad2     4334 non-null   object
 3   ad3     2949 non-null   object
 4   ad4     925 non-null    object
 5   ad5     2801 non-null   object
dtypes: object(6)
memory usage: 204.7+ KB


In [280]:
sliced_adr2_df2[sliced_adr2_df2['ad5'].notnull() == True].head()

,주소,ad1,ad2,ad3,ad4,ad5
1,광주광역시 북구 첨단벤처로108번길 9,광주광역시,북구,NaN,NaN,첨단벤처로108번길
3,전라남도 구례군 산동면 산업로 2319,전라남도,구례군,산동면,NaN,산업로
4,전라남도 구례군 산동면 노고단로 1068,전라남도,구례군,산동면,NaN,노고단로
9,전라남도 무안군 망운면 공항로 970-260,전라남도,무안군,망운면,NaN,공항로
10,전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소),전라남도,무안군,삼향읍,NaN,오룡1길


In [281]:
new_adr_idx = sliced_adr2_df2[sliced_adr2_df2['ad5'].notnull() == True].index
len(new_adr_idx)

2801

In [286]:
new_adr_idx = list(new_adr_idx)
new_adr_idx[:5]

[1, 3, 4, 9, 10]

In [298]:
len(new_adr_idx)

2801

In [291]:
total_idx = [i for i in range(4365)]
len(total_idx)

4365

In [292]:
total_idx[:5]

[0, 1, 2, 3, 4]

In [293]:
old_adr_idx = []
for i in total_idx:
    if i not in new_adr_idx:
        old_adr_idx.append(i)
len(old_adr_idx)

1564

In [295]:
sort_idx = ['old']*4365

In [296]:
for i in new_adr_idx:
    sort_idx[i] = 'new'

In [297]:
sort_idx[:5]

['old', 'new', 'old', 'new', 'new']

In [299]:
sliced_adr2_df2['주소분류'] = sort_idx
sliced_adr2_df2.head()

,주소,ad1,ad2,ad3,ad4,ad5,주소분류
0,강원도 인제군 상남면 하남리 339-7(양양방향),강원도,인제군,상남면,하남리,NaN,old
1,광주광역시 북구 첨단벤처로108번길 9,광주광역시,북구,NaN,NaN,첨단벤처로108번길,new
2,전라남도 순천시 해룡면 남가리 733-73,전라남도,순천시,해룡면,남가리,NaN,old
3,전라남도 구례군 산동면 산업로 2319,전라남도,구례군,산동면,NaN,산업로,new
4,전라남도 구례군 산동면 노고단로 1068,전라남도,구례군,산동면,NaN,노고단로,new


In [300]:
sliced_adr2_df2.to_excel('분석/2020-22년_급속충전_주소_분류.xlsx', encoding='utf-8')

# 코드 마지막